In [1]:
import pandas as pd
import glob
import os

def procesar_estaciones(file: str) -> pd.DataFrame:
    """
    Procesa un archivo Excel con hojas:
    - 'DATOS'
    - 'UBICACIÓN ESTACIONES'
    - 'ESTACIONES'

    Devuelve un DataFrame en formato largo pivotado
    con series temporales .
    """

    # === 1. Leer hojas necesarias ===
    df_datos = pd.read_excel(file, sheet_name="DATOS", engine="openpyxl")
    df_ubic = pd.read_excel(file, sheet_name="UBICACIÓN ESTACIONES", engine="openpyxl")
    df_est = pd.read_excel(file, sheet_name="ESTACIONES", engine="openpyxl")

    # === 2. Pasar de ancho a largo ===
    df_long = df_datos.melt(
        id_vars=["FDIA", "FEST", "FSEN"],
        value_vars=[f"HOR{i}" for i in range(1, 13)],
        var_name="HORA_REL",
        value_name="VALOR"
    )
    df_long["HORA_REL"] = df_long["HORA_REL"].str.extract(r"(\d+)").astype(int) - 1
    
    # === 3. Calcular hora absoluta  ===
    df_long["HORA_ABS"] = df_long.apply(
        lambda r: r["HORA_REL"] if r["FSEN"].endswith("-") else r["HORA_REL"] + 12,
        axis=1
    )

    # === 4. Extraer sentido y estación ===
    df_long["SENTIDO"] = df_long["FSEN"].str[0].astype(int)
    df_long["EST_ID"] = df_long["FEST"].str.extract(r"(\d+)").astype(int)

    # === 5. Merge con hoja de ubicacion ===
    df_merge = df_long.merge(
        df_ubic,
        left_on=["EST_ID", "SENTIDO"],
        right_on=["Estación", "Sentido"],
        how="left"
    )

    # === 6. Normaliza hoja ESTACIONES y hace merge ===
    df_est = df_est.rename(columns={
        "Nº": "Estación",
        "ESTACIÓN": "Nombre_est",
        "LATITUD": "Lat_base",
        "LONGITUD": "Lon_base"
    })
    df_merge = df_merge.merge(
        df_est[["Estación", "Nombre_est", "Lat_base", "Lon_base"]],
        on="Estación",
        how="left"
    )

    # === 7. Construir nombre de columna ===
    df_merge["COL_NAME"] = df_merge["Nombre"] + "_" + df_merge["Orient."]

    # === 8. Transforma dats de fecha a tipo timestamp ===
    df_merge["FDIA"] = pd.to_datetime(df_merge["FDIA"], dayfirst=True)
    df_merge["timestamp"] = df_merge["FDIA"] + pd.to_timedelta(df_merge["HORA_ABS"], unit="h")

    print(df_merge.head())

    # === 9. Pivotar series temporales ===
    tabla_valores = df_merge.pivot_table(
        index="timestamp",
        columns="COL_NAME",
        values="VALOR"
    ).reset_index()

    # === 10. Redondear a enteros ===
    num_cols = tabla_valores.select_dtypes(include=["float", "float64", "float32"]).columns
    tabla_valores[num_cols] = tabla_valores[num_cols].round(0).astype("Int64")

    return tabla_valores


def unir_estaciones(carpeta: str) -> pd.DataFrame:
    """
    Recorre la carpeta buscando archivos que empiezan con 'DATOS_ESTACIONES',
    concatna los resultados y los ordena por fecha.
    """
    # Buscar todos los archivos que empiezan por DATOS_ESTACIONES
    archivos = sorted(glob.glob(os.path.join(carpeta, "DATOS_ESTACIONES*.xlsx")))

    if not archivos:
        raise FileNotFoundError("No se encontraron archivos 'DATOS_ESTACIONES*.xlsx' en la carpetaindicada.")

    lista_df = []
    for archivo in archivos:
        print(f"Procesando: {os.path.basename(archivo)}")
        df_temp = procesar_estaciones(archivo)
        lista_df.append(df_temp)

    # Concatenar todos los meses
    df_final = pd.concat(lista_df, ignore_index=True)

    # Ordenar por fecha
    df_final = df_final.sort_values("timestamp").reset_index(drop=True)

    return df_final


# === Ejecutar ===
datafinal = unir_estaciones("./DatosEstacionesOctubre2021-Marzo2025")
datafinal.tail()
datafinal.to_excel("DF_FINAL.xlsx", index=False)


Procesando: DATOS_ESTACIONES_ABRIL_2022.xlsx
        FDIA  FEST FSEN  HORA_REL  VALOR  HORA_ABS  SENTIDO  EST_ID  Estación  \
0 2022-04-01  ES01   1-         0    518         0        1       1       1.0   
1 2022-04-01  ES01   1=         0   2161        12        1       1       1.0   
2 2022-04-01  ES01   2-         0    413         0        2       1       1.0   
3 2022-04-01  ES01   2=         0   1329        12        2       1       1.0   
4 2022-04-01  ES02   1-         0    285         0        1       2       2.0   

                   Nombre           Latitud           Longitud  Sentido  \
0  Paseo de la Castellana  40,4319272588958  -3,68910874956933      1.0   
1  Paseo de la Castellana  40,4319272588958  -3,68910874956933      1.0   
2  Paseo de la Castellana  40,4319272588958  -3,68910874956933      2.0   
3  Paseo de la Castellana  40,4319272588958  -3,68910874956933      2.0   
4          Calle Princesa  40,4260679064732  -3,71270896289063      1.0   

  Orient.        

C:\Users\patri_ey1tp50\AppData\Local\Temp\ipykernel_10976\3918316004.py:65: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_merge["FDIA"] = pd.to_datetime(df_merge["FDIA"], dayfirst=True)


        FDIA  FEST FSEN  HORA_REL  VALOR  HORA_ABS  SENTIDO  EST_ID  Estación  \
0 2021-12-01  ES01   1-         0    320         0        1       1       1.0   
1 2021-12-01  ES01   1=         0   1951        12        1       1       1.0   
2 2021-12-01  ES01   2-         0    196         0        2       1       1.0   
3 2021-12-01  ES01   2=         0   1581        12        2       1       1.0   
4 2021-12-01  ES02   1-         0    288         0        1       2       2.0   

                   Nombre           Latitud           Longitud  Sentido  \
0  Paseo de la Castellana  40,4319272588958  -3,68910874956933      1.0   
1  Paseo de la Castellana  40,4319272588958  -3,68910874956933      1.0   
2  Paseo de la Castellana  40,4319272588958  -3,68910874956933      2.0   
3  Paseo de la Castellana  40,4319272588958  -3,68910874956933      2.0   
4          Calle Princesa  40,4260679064732  -3,71270896289063      1.0   

  Orient.              Nombre_est   Lat_base  Lon_base  \
0   

C:\Users\patri_ey1tp50\AppData\Local\Temp\ipykernel_10976\3918316004.py:65: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_merge["FDIA"] = pd.to_datetime(df_merge["FDIA"], dayfirst=True)


        FDIA  FEST FSEN  HORA_REL  VALOR  HORA_ABS  SENTIDO  EST_ID  Estación  \
0 2022-12-01  ES01   1-         0    116         0        1       1       1.0   
1 2022-12-01  ES01   1=         0   2071        12        1       1       1.0   
2 2022-12-01  ES01   2-         0    209         0        2       1       1.0   
3 2022-12-01  ES01   2=         0   1212        12        2       1       1.0   
4 2022-12-01  ES02   1-         0    222         0        1       2       2.0   

                   Nombre           Latitud           Longitud  Sentido  \
0  Paseo de la Castellana  40,4319272588958  -3,68910874956933      1.0   
1  Paseo de la Castellana  40,4319272588958  -3,68910874956933      1.0   
2  Paseo de la Castellana  40,4319272588958  -3,68910874956933      2.0   
3  Paseo de la Castellana  40,4319272588958  -3,68910874956933      2.0   
4          Calle Princesa  40,4260679064732  -3,71270896289063      1.0   

  Orient.              Nombre_est   Lat_base  Lon_base  \
0   